# Creating and  Using Graph DB in Python Application

## Prepare the Environment

In [217]:
from neo4j import GraphDatabase, RoutingControl, basic_auth

In [218]:
import pandas as pd
import numpy as np
#fewkfew

## Setup the Connection to the Database
Choose a database server, a database, credentials for accessing it, and use them to configure the driver.\
The driver helps connecting the database and passing the Cypher statements

In [219]:
# DB URI and authentication
URI = "bolt://localhost:7687"
AUTH = ("neo4j","mikkel405")

In [220]:
driver = GraphDatabase.driver(URI, auth=AUTH)

In [221]:
driver.verify_connectivity()

### Query Templates

Execute a Cypher statement by creating a session and using the methods 
- Session.execute_read() and 
- Session.execute_write()

Do not hardcode or concatenate parameters: 
- use placeholders and 
- specify the parameters as keyword arguments

In [222]:
# template write request
def write_query(query, params={}):
    with driver.session(database="neo4j") as session:
        results = session.execute_write(lambda tx: tx.run(query, params).data())
        # response = [record.values()[0] for record in results]
        return results

In [223]:
# template read request
def read_query(query, params={}):
    with driver.session(database="neo4j") as session:
        results = session.execute_read(lambda tx: tx.run(query, params).data())       
        response = [record.values()[0] for record in results]
        return response

## Harry Potter Demo Implementation

### File Reading Utilities

In [224]:
data_dir = '../Data/HarryPotter/'
file_name = 'HP-characters.csv'
path = data_dir+file_name

In [225]:
# with pandas
data = pd.read_csv(path)
data

,title,url,loyalty,family,blood,nationality,species,house,gender,aliases
0,Vernon Dursley,https://harrypotter.fandom.com/wiki/Vernon_Dur...,"Dursley family,Grunnings","Mrs Dursley,Marjorie Dursley,Petunia Dursley,D...",Muggle,English,Human,NaN,Male,NaN
1,Petunia Dursley,https://harrypotter.fandom.com/wiki/Petunia_Du...,"Evans family,Dursley family","Mr Evans,Mrs Evans,Lily Potter,Vernon Dursley,...",Muggle,English,Human,NaN,Female,[]
2,Dudley Dursley,https://harrypotter.fandom.com/wiki/Dudley_Dur...,"Dursley family,Evans family,Smeltings Academy,...","Vernon Dursley,Petunia Dursley,Mrs Dursley,Chi...",Muggle,English,Human,NaN,Male,"['Dudders', 'Ickle Dudleykins', 'Popkin', 'Did..."
3,Lily J. Potter,https://harrypotter.fandom.com/wiki/Lily_J._Po...,"Evans family,Potter family,Hogwarts School of ...","Mr Evans,Mrs Evans,Petunia Dursley,James Potte...",Muggle-born,English,Human,Gryffindor,Female,NaN
4,James Potter I,https://harrypotter.fandom.com/wiki/James_Pott...,"Potter family,Evans family,Hogwarts School of ...","Ignotus Peverell,Linfred of Stinchcombe,Hardwi...",Pure-blood,English,Human,Gryffindor,Male,[]
...,...,...,...,...,...,...,...,...,...,...
157,Elfric the Eager,https://harrypotter.fandom.com/wiki/Elfric_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,Hogwarts Giant Squid,https://harrypotter.fandom.com/wiki/Hogwarts_G...,NaN,NaN,NaN,NaN,Giant Squid,NaN,NaN,NaN
159,Quirinus Quirrell's second mountain troll,https://harrypotter.fandom.com/wiki/Quirinus_Q...,NaN,NaN,NaN,NaN,Mountain Troll,NaN,NaN,NaN
160,King's Cross Station,https://harrypotter.fandom.com/wiki/King%27s_C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [226]:
data.isnull().sum()

title            0
url              0
loyalty         63
family         106
blood           79
nationality     53
species         10
house           94
gender          19
aliases        121
dtype: int64

In [227]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        162 non-null    object
 1   url          162 non-null    object
 2   loyalty      99 non-null     object
 3   family       56 non-null     object
 4   blood        83 non-null     object
 5   nationality  109 non-null    object
 6   species      152 non-null    object
 7   house        68 non-null     object
 8   gender       143 non-null    object
 9   aliases      41 non-null     object
dtypes: object(10)
memory usage: 12.8+ KB


In [228]:
datax = data.replace({np.nan: None})

In [229]:
datax.sample(5)

,title,url,loyalty,family,blood,nationality,species,house,gender,aliases
158,Hogwarts Giant Squid,https://harrypotter.fandom.com/wiki/Hogwarts_G...,None,None,None,None,Giant Squid,None,None,None
18,Arabella Figg,https://harrypotter.fandom.com/wiki/Arabella_Figg,"Albus Dumbledore,Order of the Phoenix,Figg family",None,Squib,British,Human,None,Female,None
98,Peeves,https://harrypotter.fandom.com/wiki/Peeves,"Hogwarts School of Witchcraft and Wizardry,Ord...",None,None,None,Poltergeist,None,Male,"['Peevsie', 'Peevesy']"
50,Newton Scamander,https://harrypotter.fandom.com/wiki/Newton_Sca...,"Scamander family,Goldstein family,Hogwarts Sch...","Mrs Scamander,Theseus Scamander,Porpentina Gol...",Pure-blood,English,Human,Hufflepuff,Male,['Newt']
99,Sorting Hat,https://harrypotter.fandom.com/wiki/Sorting_Hat,None,None,None,None,None,None,None,None


In [230]:
datax['house'].tolist()

[None,
 None,
 None,
 'Gryffindor',
 'Gryffindor',
 'Gryffindor',
 None,
 'Slytherin',
 None,
 None,
 None,
 'Gryffindor',
 'Gryffindor',
 None,
 None,
 'Gryffindor',
 'Gryffindor',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'Slytherin',
 None,
 None,
 None,
 'Hufflepuff',
 'Gryffindor',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'Hufflepuff',
 None,
 None,
 None,
 'Ravenclaw',
 None,
 None,
 None,
 'Slytherin',
 'Slytherin',
 'Slytherin',
 None,
 None,
 'Ravenclaw',
 None,
 'Gryffindor',
 'Gryffindor',
 'Gryffindor',
 'Gryffindor',
 'Gryffindor',
 'Gryffindor',
 'Gryffindor',
 None,
 'Gryffindor',
 None,
 'Gryffindor',
 'Gryffindor',
 'Gryffindor',
 'Gryffindor',
 None,
 None,
 None,
 None,
 None,
 None,
 'Hufflepuff',
 None,
 None,
 None,
 None,
 None,
 None,
 'Gryffindor',
 'Slytherin',
 'Slytherin',
 None,
 None,
 'Hufflepuff',
 None,
 None,
 'Hufflepuff',
 'Hufflepuff',
 'Raven

### Ingest the Data into the Database

#### Characters

In [231]:
create_db = """
MERGE (c:Character{name:$title, url:$url})
    SET c.aliases = CASE trim(toString(c.aliases)) WHEN 'None' THEN null ELSE $aliases END
    SET c.blood = CASE trim(toString(c.blood)) WHEN 'None' THEN null ELSE $blood END
    SET c.nationality = CASE trim(toString(c.nationality)) WHEN 'None' THEN null ELSE $nationality END
    SET c.gender = CASE trim(toString(c.gender)) WHEN 'None' THEN null ELSE $gender END
    SET c.species = CASE trim(toString(c.species)) WHEN 'None' THEN null ELSE $species END
  
"""


In [232]:
# Execute transaction
for index, row in datax.iterrows():
    response = write_query(create_db, 
                           params = {'title':row['title'], 'url':row['url'], 'aliases':row['aliases'],
                                     'blood':row['blood'], 'nationality':row['nationality'], 
                                     'species':row['species'], 'gender':row['gender'] 
                                    })

In [233]:
add_groups = '''
    MERGE (c:Character {title:$title})
    WITH c
    UNWIND split($loyalty, ',') AS l
        MERGE (g:Group {name:l}) 
        MERGE (c)-[:LOYAL_TO]->(g)
    RETURN c
'''

In [234]:
# Execute transaction
for index, row in datax.iterrows():
    response = write_query(add_groups, params = {'title':row['title'], 'loyalty':row['loyalty'] })

In [235]:
add_family = '''
    MERGE (c:Character {title:$title})
    WITH c
    UNWIND split($family, ',') AS f
    MERGE (f1:Character {name:f}) 
    MERGE (c)-[t:IN_FAMILY_WITH]->(f1) 
    RETURN c
'''

In [236]:
# Execute transaction
for index, row in datax.iterrows():
    response = write_query(add_family, params = {'title':row['title'], 'family':row['family'] })

In [237]:
add_house = '''
    MERGE (c:Character {title:$title})
    WITH c
    FOREACH (h in CASE WHEN $house IS NOT NULL THEN [1] ELSE [] END | 
    MERGE (h1:House {name:$house}) 
    MERGE (c)-[:BELONGS_TO]->(h1)  ) 
    RETURN c
'''

In [238]:
# Execute transaction
for index, row in datax.iterrows():
    response = write_query(add_house, params = {'title':row['title'],'house':row['house']})

#### Relations

In [239]:
file_name = 'HP-relations-seen.csv'
path = data_dir+file_name

In [240]:
# with pandas
data = pd.read_csv(path)
data

,source,target,value
0,Petunia Dursley,Vernon Dursley,67629
1,Dudley Dursley,Vernon Dursley,30647
2,Dudley Dursley,Petunia Dursley,104637
3,Harry Potter,Vernon Dursley,30959
4,Harry Potter,Petunia Dursley,104646
...,...,...,...
401,George Weasley,Poppy Pomfrey,100216
402,Nicolas Flamel,Perenelle Flamel,100608
403,Albus Dumbledore,Helena Ravenclaw,103872
404,Ronald Weasley,Vernon Dursley,104671


In [241]:
data.isnull().sum()

source    0
target    0
value     0
dtype: int64

In [242]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  406 non-null    object
 1   target  406 non-null    object
 2   value   406 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 9.6+ KB


In [243]:
datay = data.replace({np.nan: None})

In [244]:
datay.sample(5)

,source,target,value
375,Ronan,Rubeus Hagrid,85951
32,Mr Paws,Tufty,8093
355,Gregory Goyle,Neville Longbottom,104122
15,Harry Potter,Lily J. Potter,18611
364,Poppy Pomfrey,Ronald Weasley,101898


In [245]:
create_rel = """
    MERGE (s:Character{name:$source})
    MERGE (t:Character{name:$target})
    MERGE (s)-[r:INTERACTS]-(t)
    SET r.weight = $value
"""    

In [246]:
# Execute transaction
for index, row in datay.iterrows():
    response = write_query(create_rel, 
                           params = {'source':row['source'], 'target':row['target'], 'value':row['value']})

## Query the Database

To enable permissions for running the algorithms on the localmachine, aadd this line to your config file:
__dbms.security.procedures.unrestricted=jwt.security.*, apoc.*, gds.*__

In [247]:
def read_gds(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

### Community Detection

In [248]:
# create a projection
query = """
    CALL gds.graph.project('dor', 'Character', {INTERACTS:{orientation:'UNDIRECTED'}})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS title, score AS rank
    ORDER BY rank DESCENDING, title LIMIT 5
"""

In [249]:
response = read_gds(query)
response

ClientError: {code: Neo.ClientError.Procedure.ProcedureNotFound} {message: There is no procedure with the name `gds.graph.project` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.}

In [ ]:
query = """
    CALL gds.louvain.stream('dom')
    YIELD nodeId, communityId, intermediateCommunityIds
    RETURN gds.util.asNode(nodeId).name AS name, communityId
    ORDER BY communityId ASC LIMIT 15
"""

In [ ]:
# writes the calculated community coeficients as parameters of the nodes
response = read_gds(query)
response

### Degree Centrality

In [ ]:
query = """
    CALL gds.degree.stream('dor')
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS title, score AS connections
    ORDER BY rank DESCENDING, title LIMIT 5
"""

In [ ]:
response = read_gds(query)
response

### Page Rank

In [ ]:
# create projection
query = '''
CALL gds.graph.project(
  'dom',
  'Character',
  'INTERACTS',
  {
    relationshipProperties: 'weight'
  }
)
'''

In [ ]:
response = read_gds(query)
response

In [ ]:
query = """
    CALL gds.pageRank.stream('dom')
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).title AS title, score
    ORDER BY score DESC
"""

In [ ]:
response = read_gds(query)
response

### Shortest Path

In [ ]:
# create projection
query = '''
CALL gds.graph.project(
    'pat',
    'Character',
    'INTERACTS',
    {
        relationshipProperties: 'weight'
    }
)
'''

In [ ]:
response = read_gds(query)
response

In [ ]:
query = """
    MATCH (source:Character {title: 'Harry Potter'}), (target:Character {title: 'Albus Dumbledore'})
    CALL gds.shortestPath.dijkstra.stream('pat', 
    {
        sourceNode: source,
        targetNode: target,
        relationshipWeightProperty: 'weight'
    })
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        index,
        gds.util.asNode(sourceNode).title AS sourceNodeName,
        gds.util.asNode(targetNode).title AS targetNodeName,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).title] AS nodeNames,
        costs,
        nodes(path) as path
    ORDER BY index
"""

In [ ]:
response = read_gds(query)
response